In [0]:
SET TIME ZONE 'Pacific/Auckland';

In [0]:
SELECT TradingDate, TradingPeriodNumber,
                Case 
                  -- When dat.PointOfConnectionCode in ('GLN0332','KAW2201') Then 'NI' 
                  -- When dat.PointOfConnectionCode in ('INV0661') Then 'SI' 
                  When dat.GridZoneCode = 'GZ8' Then 'HAY' 
                  Else dat.IslandCode 
                End as Region,
                sum(KilowattHours)/500 as MW 
                FROM datahub.wholesale.reconciledinjectionandofftake dat 
                WHERE dat.FlowDirection = 'Offtake'
                AND TradingDate >= '2025-09-01'
                GROUP BY all 
                ORDER BY TradingDate, TradingPeriodNumber, Region

In [0]:
select *
from ea_prd.gold.factspdbranches where TradingDate = '2025-10-23' and TradingPeriodNumber >= 14
and CaseTypeCode = 'RTD' and SPDBranchMarginalDollarsPerMegawattHour > 0 



In [0]:
select *
from ea_prd.gold.factspdmarketnodeconstraints where TradingDate = '2025-10-23'
and CaseTypeCode = 'RTD' and (SPDMarketNodeConstraintName like 'DN_STK%' or SPDMarketNodeConstraintName like 'DN_ARG%')
order by IntervalDateTime

In [0]:
select *
from ea_prd.gold.factthermalfueldisclosurestransactionsgas
where ParticipantCode = 'GENE'
and SubmissionDate > '2025-10-01'
order by SubmissionDate, Date

In [0]:
select * from ea_prd.bronze.thermalfueldisclosuresstoragediesel

In [0]:
select * from ea_prd.gold.factthermalfueldisclosuresstoragediesel

In [0]:
select UpdatedDate, count(*) from ea_prd.silver.spd_solution_constraint group by all


In [0]:
with L1 as (
  select UpdatedDate, count(*) as silver 
  from ea_prd.silver.spd_mssdata_branchconstrfactors  
  group by all
)
, L2 as (
  select UpdatedDate, count(*) as gold 
  from ea_prd.gold.factspdmarketnodeconstraints
  group by all
)
select L1.*, L2.gold from L1 left join L2 on L1.UpdatedDate = L2.UpdatedDate --and L1.CaseID = L2.CaseID
where L1.silver <> L2.gold or L2.gold is null

In [0]:
select CaseID, count(*) from  
from ea_prd.silver.spd_solution_branch
group by all

In [0]:
select CaseID, count(*) from  
from ea_prd.gold.factspdbranches
group by all

In [0]:
with L1 as (select CaseID, IntervalDateTime, PNodeCode, avg(SPDGenerationMegawatts) as Cleared_MW 
from ea_prd.gold.factspdbidsandoffers 
where Tradingdate = '2025-08-21' 
and TradingPeriodNumber = 30 
and CaseTypeCode = 'RTD'
and Interval = '21-AUG-2025 14:30'
and OfferTypeCode = 'ENOF'
Group by CaseID, IntervalDateTime, PNodeCode)
, L2 as (
  select CaseID, IntervalDateTime, sum(Cleared_MW) as Cleared_MW 
  from L1 Group by CaseID, IntervalDateTime)
, L3 as (
  select SPDOfferPriceDollarsPerMegawattHour as Offer_Price, 
  case 
    when SPDOfferOutputMegawatts >= 9999 then SPDAdjustedOfferOutputMegawatts
    else SPDOfferOutputMegawatts 
  end as Offer_MW, 
  SPDAdjustedOfferOutputMegawatts as Adjuted_Offer_MW, 
  Cleared_MW  
  from ea_prd.gold.factspdbidsandoffers a INNER JOIN L2 ON 1=1
where Tradingdate = '2025-08-21' 
and TradingPeriodNumber = 30 
and CaseTypeCode = 'RTD'
and Interval = '21-AUG-2025 14:30'
and OfferTypeCode = 'ENOF')
select Offer_Price,	sum(Offer_MW) as Offer_MW,
sum(Adjuted_Offer_MW) as Adjusted_MW,
avg(Cleared_MW) as Cleared_MW
from L3
Group by Offer_Price
Order by Offer_Price

In [0]:
select count(*) from ea_prd.bronze.spd_solution_branch

In [0]:
select ParticipantCode, MonthEnded, 
ContractedGasMinPetajoulesPerMonth * 1e6 as Monthly_Min_GJ,
ContractedGasMaxPetajoulesPerMonth * 1e6 as Monthly_Max_GJ,
DeliverabilityMinTerajoulesPerDay * 1e3 as Daily_Max_GJ,	
DeliverabilityMaxTerajoulesPerDay * 1e3 as Daily_Min_GJ,	
SubmissionDate
from  datahub.wholesale.thermalfueldisclosurescontractsgas
order by ParticipantCode, MonthEnded

In [0]:
select * from datahub.wholesale.thermalfueldisclosuresstoragegas

In [0]:
select * from  ea_prd.gold.factthermal
where ParticipantCode='CTCT' and DateSupplied = '2025-08-10'

In [0]:
select 
ParticipantCode, DateSupplied, TransactionType, max(PriceExclCarbonDollarsPerGigajoule),
round(sum(VolumeGigajoules*PriceExclCarbonDollarsPerGigajoule)/sum(VolumeGigajoules),2) as VWAP
from ea_prd.gold.factthermalfueldisclosurestransactionsgas
where TRansactionType in ('Sale','Purchase') and DateSupplied >= '2025-01-01'
and VolumeGigajoules > 0 
Group BY all
order by all

In [0]:
select TRadingDate,GasVWAPDollarsPerGigajoule, * from datahub.wholesale.gasandcarbonpriceindices 
where TRadingDate>= '2025-01-01'
order by TRadingDate
